Export all FEFLOW Kx values as Center in an ASCII table and read the table into a dataframe with pandas

In [102]:
import pandas as pd
import numpy as np

#Import dask for parallelization of merge
import dask
import dask.dataframe as dd

feflow = pd.read_fwf('Carrot_Kx.dat', colspecs = 'infer', infer_nrows = 12)
feflow.head(5)

,Id,X,Y,Z,M,Element,CENTER_X,CENTER_Y,CENTER_Z,LAYER,CONDX,CONDX_U
0,1,619465.140152,5.854999e+06,714.518971,0.0,1,619465.140152,5.854999e+06,714.518971,1,0.000012,m/s
1,2,619476.500000,5.854990e+06,714.289886,0.0,2,619476.500000,5.854990e+06,714.289886,1,0.000012,m/s
2,3,619470.140152,5.855018e+06,714.366445,0.0,3,619470.140152,5.855018e+06,714.366445,1,0.000012,m/s
3,4,619461.973485,5.855031e+06,714.118436,0.0,4,619461.973485,5.855031e+06,714.118436,1,0.000012,m/s
4,5,611400.399115,5.856187e+06,637.554506,0.0,5,611400.399115,5.856187e+06,637.554506,1,0.000012,m/s


Append new indexing column equal to the sum of X,Y,Z truncated to 0 decimal places

In [ ]:
import math as math

#Truncate values in feflow dataframe
feflow_trunc = feflow[['X','Y','Z']].copy()
feflow_trunc['X'] = feflow_trunc['X'].astype(int)
feflow_trunc['Y'] = feflow_trunc['Y'].astype(int)
feflow_trunc['Z'] = feflow_trunc['Z'].astype(int)
print(feflow_trunc.head(5))

#FEFLOW SUM INDEX
fe_col_list = list(feflow_trunc)
feflow_sum = feflow_trunc[fe_col_list].sum(axis=1)
feflow["Sum"]=feflow_sum
feflow.head(5)

Import RQD Block Model as CSV

In [105]:
mfq = dd.read_csv('MFQ_RQD.csv')
mfq.head(5)

,Id,X,Y,Z,dX,dY,dZ,RQD%
0,291275,612065,5853415,-59,10,10,14,98.718974
1,291276,612075,5853415,-59,10,10,14,98.719645
2,291277,612085,5853415,-59,10,10,14,98.720326
3,291278,612095,5853415,-59,10,10,14,98.721018
4,291279,612105,5853415,-59,10,10,14,98.721720


Define mathematical transformation to convert RQD values to hydraulic conductvity

In [ ]:
def transform(x):
    return x/1000 

Create New Dataframe (mfqkx) and apply transformation of RQD values to Kx on MFQ dataframe

In [ ]:
import numpy as np
mfq_transform = mfq.apply(lambda x: transform(x) if x.name == 'RQD%' else x)
mfqkx = mfq_transform[['X','Y','Z']].copy()

#Add sum index to dataframe
mfqkx['X'] = mfqkx['X'].astype(int)
mfqkx['Y'] = mfqkx['Y'].astype(int)
mfqkx['Z'] = mfqkx['Z'].astype(int)
#MFQKX SUM INDEX
mfqkx_col_list = list(mfqkx)
mfqkx_sum = mfqkx[fe_col_list].sum(axis=1)
mfqkx["Sum"] = mfqkx_sum
mfqkx['RQD'] = pd.Series(mfq_transform['RQD%'])
print(mfqkx.head(5))


In [ ]:
inner_merge = dd.merge(mfqkx, feflow, on=['Sum'])
print(inner_merge.head(5))

Export final dataframe to .xlsx (Excel) file that can be read for parameter linkage in FEFLOW

In [ ]:
#table_import.to_excel('test.xlsx')